---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
a1=df.iloc[:125]
a1=a1.str.extractall(r'((?P<month>\d?\d)[/-](?P<date>\d?\d)[/-](?P<year>\d{2,4}))')
a1=a1.reset_index()
a1=a1[["month","date","year"]]
a1["year"]=a1["year"].apply(lambda x: "19"+x if len(x)==2 else x)
a1["date"]=a1["date"].apply(lambda x: int(x[1:]) if x[0]==str(0) else int(x))
a1["month"]=a1["month"].apply(lambda x: x[1:] if x[0]==str(0) else x)
a1=a1[a1["date"]<32]
a1["date"]=a1["date"].apply(lambda x:str(x))
a1=a1.iloc[:,0:3].apply(lambda x: '/'.join(x), axis=1)
print(a1[:5],a1.shape)

0    3/25/1993
1    6/18/1985
2     7/8/1971
3    9/27/1975
4     2/6/1996
dtype: object (125,)


In [3]:
a2=df.iloc[125:194]
a2=a2.str.extractall(r'((?P<date>\d\d)[\s](?P<month>\w{3,10}.?)[\s](?P<year>\d{4}))')
a2=a2.reset_index()
a2=a2[["month","date","year"]]
def month_converter(month):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return str(months.index(month) + 1)
a2["month"]=a2["month"].apply(lambda x:month_converter(x[:3]))
a2=a2.iloc[:,0:3].apply(lambda x: '/'.join(x), axis=1)
print(a2[:5],a2.shape)

0    1/24/2001
1    9/10/2004
2    5/26/1982
3    6/28/2002
4    5/06/1972
dtype: object (69,)


In [4]:
a3=df.iloc[194:228]
a3=a3.str.extractall(r'((?P<month>\w{3,10}.?)[\s](?P<date>\d{1,2},?)[\s](?P<year>\d{4}))')
a3=a3.reset_index()
a3=a3[["month","date","year"]]
a3["month"]=a3["month"].apply(lambda x:month_converter(x[:3]))
a3["date"]=a3["date"].apply(lambda x:x[:2])
a3=a3.iloc[:,0:3].apply(lambda x: '/'.join(x), axis=1)
print(a3[:5],a3.shape)

0     4/11/1990
1     5/30/2001
2     2/18/1994
3     2/18/1981
4    10/11/2013
dtype: object (34,)


In [5]:
a4=df.iloc[228:343]
a4=a4.str.extractall(r'((?P<month>[A-Za-z]{2,10}.?)[\s](?P<year>\d{4}\b))')
a4=a4.reset_index()
a4["date"]=str(1)
a4=a4[["month","date","year"]]
a4["month"]=a4["month"].apply(lambda x: month_converter(x[1:4]) if x[0].islower()==True or x[0]=="I" or x[:2]=="AF" else month_converter(x[:3]))
a4=a4.iloc[:,0:3].apply(lambda x: '/'.join(x), axis=1)
print(a4[:5],a4.shape)

0    9/1/1985
1    6/1/2011
2    5/1/1986
3    5/1/2016
4    7/1/1977
dtype: object (115,)


In [6]:
a5=df.iloc[343:455]
a5=a5.str.extractall(r'((?P<month>\d{1,3})[/](?P<year>\d{4}\b))')
a5=a5.reset_index()
a5["date"]=str(1)
a5=a5[["month","date","year"]]
a5["month"]=a5["month"].apply(lambda x: x[1:] if x[0]==str(0) else x)
a5=a5.iloc[:,0:3].apply(lambda x: '/'.join(x), axis=1)
print(a5[:5],a5.shape)

0     6/1/1998
1     6/1/2005
2    10/1/1973
3     9/1/2005
4     3/1/1980
dtype: object (112,)


In [7]:
a6=df.iloc[455:]
print(a6.shape)
a6=a6.str.extractall(r'((?P<year>\d{4}\b))')
a6=a6.reset_index()
a6["month"]=str(1)
a6["date"]=str(1)
a6=a6[["month","date","year"]]
a6=a6.iloc[:,0:3].apply(lambda x: '/'.join(x), axis=1)
print(a6[:5],a6.shape)

(45,)
0    1/1/1984
1    1/1/2000
2    1/1/2001
3    1/1/1982
4    1/1/1998
dtype: object (45,)


In [8]:
final=pd.concat([a1,a2,a3,a4,a5,a6],axis=0)
final=pd.DataFrame(final)
final.index=range(len(final))
final.columns=["time"]
final["time"]=final["time"].apply(lambda x:pd.to_datetime(x, infer_datetime_format=True) )
final=final.sort_values("time",ascending=False)
final

,time
413,2016-11-01
161,2016-10-19
186,2016-10-13
141,2016-05-30
231,2016-05-01
427,2016-05-01
253,2016-02-01
464,2016-01-01
235,2015-10-01
152,2015-09-28
